# LangGraph Tutorial: Setup & Validation

**What You'll Build:** A financial assistant agent with currency conversion and EMI calculation capabilities.

**This Notebook:** Environment setup and connectivity validation.

## Required Environment Variables

Create a `.env` file with:

```env
# Gemini AI Configuration
GOOGLE_API_KEY=<api key from aistudio.google.com>

# LangSmith Observability (Optional)
LANGSMITH_API_KEY=your-langsmith-key
LANGSMITH_PROJECT=langgraph-tutorial
LANGCHAIN_TRACING_V2=true
```

**Installation:**
```bash
pip install langchain langchain-google-genai langgraph python-dotenv
```

## Import Dependencies

In [1]:
# LLM Provider
from langchain_google_genai import ChatGoogleGenerativeAI

# Environment management
import os
from dotenv import load_dotenv

print("✅ All imports successful")

✅ All imports successful


## Load Environment Variables

In [2]:
# Load environment variables from .env file
load_dotenv("../../.env")  # Adjust path to your .env location

True

In [3]:
print("✅ Environment loaded successfully")
print(f"   Project: {os.getenv('GOOGLE_PROJECT_ID')}")
print(f"   Region: {os.getenv('GOOGLE_REGION')}")
print(f"   LangSmith Tracing: {'Enabled' if os.getenv('LANGCHAIN_TRACING_V2') else 'Disabled'}")

✅ Environment loaded successfully
   Project: None
   Region: None
   LangSmith Tracing: Disabled


## Initialize LLM

In [4]:
# Create the base LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-pro",
    temperature=0.3,  # Lower temperature for consistent, factual responses
    max_tokens=1024,
    # project=os.getenv("GOOGLE_PROJECT_ID"),  
    # location=os.getenv("GOOGLE_REGION")
)

print("✅ LLM initialized successfully")
print(f"   Model: gemini-2.5-pro")
print(f"   Temperature: 0.3")
print(f"   Max Tokens: 1024")

✅ LLM initialized successfully
   Model: gemini-2.5-pro
   Temperature: 0.3
   Max Tokens: 1024


## Test LLM Connectivity

In [5]:
# Simple test to verify LLM is working
test_response = llm.invoke("Say 'Hello from LangGraph!' if you can read this.")

print("🧪 LLM Test:")
print(f"   Response: {test_response.content}")
print("\n✅ LLM connectivity verified!")

🧪 LLM Test:
   Response: Hello from LangGraph

✅ LLM connectivity verified!


---

## ✅ Setup Complete!

**Verified:**
- ✅ Environment variables loaded
- ✅ Dependencies imported
- ✅ LLM connectivity working

**Next:** Notebook 2 - Tools Creation